## Description

This notebook will extract training data from the ODC using geometries within a geojson. The dataset will use the NNI level labels within the 'data/nni_training_spain.geojson' file.

In [1]:
%matplotlib inline

import os
import datacube
import numpy as np
import geopandas as gpd
from datacube.utils.geometry import assign_crs

import sys
sys.path.insert(1, '../')
from tools.plotting import map_shapefile
from tools.bandindices import calculate_indices
from tools.datahandling import mostcommon_crs
from tools.classification import collect_training_data

import warnings
warnings.filterwarnings("ignore")

In [2]:
#Connect to the datacube
dc = datacube.Datacube(app='Sentinel-2')

## Analisis parameters
* path : The path to the input vector file from witch we wil extract training data.
* field : This is the name of the columns in your shapefile attribute table that contains the class lables. The class lables must be integers.

In [3]:
path = 'data/nni_training_egypt.geojson' 
field = 'class'

## Find the number of CPUs

In [4]:
#ncpus=round(get_cpu_quota()) Calculate the number of cpus we set it to 1
ncpus = 1
print('ncpus = '+str(ncpus))

ncpus = 1


# Preview input data
We can load and preview our input data shapefile using geopandas.

In [5]:
# Load input data shapefile
input_data = gpd.read_file(path)

# Plot first five rows
input_data.head()

,class,geometry
0,4,"POLYGON ((30.71267 30.50485, 30.71085 30.50450..."
1,3,"POLYGON ((31.97251 30.45663, 31.97206 30.45661..."
2,3,"POLYGON ((31.96894 30.46447, 31.96850 30.46445..."
3,2,"POLYGON ((31.92517 30.44214, 31.92481 30.44212..."
4,2,"POLYGON ((31.92315 30.45008, 31.92272 30.45006..."


In [6]:
# Plot training data in an interactive map
map_shapefile(input_data, attribute=field)

Label(value='')

Map(center=[30.823458212027198, 31.263168430040523], controls=(ZoomControl(options=['position', 'zoom_in_text'…

# Extracting training data
The function collect_training_data takes our geojson containing class labels and extracts training data (features) from the datacube over the locations specified by the input geometries. The function will also pre-process our training data by stacking the arrays into a useful format and removing any NaN or inf values.The below variables can be set within the collect_training_data function:

* zonal_stats: An optional string giving the names of zonal statistics to calculate across each polygon (if the geometries in the vector file are polygons and not points). Default is None (all pixel values are returned). Supported values are 'mean', 'median', 'max', and 'min'.

In addition to the zonal_stats parameter, we also need to set up a datacube query dictionary for the Open Data Cube query such as measurements (the bands to load from the satellite), the resolution (the cell size), and the output_crs (the output projection). These options will be added to a query dictionary that will be passed into collect_training_data using the parameter collect_training_data(dc_query=query, ...). The query dictionary will be the only argument in the feature layer function which we will define and describe in a moment.

In [7]:
#set up our inputs to collect_training_data
zonal_stats = 'mean'

# Set up the inputs for the ODC query
# Create a reusable query
query = {
    'time': ('2022'),
    'resolution': (-20, 20),
    'measurements': ['red', 'green', 'red_edge_1', 'red_edge_2', 'red_edge_3', 'nir']
}

# Identify the most common projection system in the input query
output_crs = mostcommon_crs(dc=dc, product='gm_s2_annual', query=query)
print(output_crs)

query.update({"output_crs": output_crs})

epsg:6933


## Defining feature layers
To create the desired feature layers, we pass instructions to collect_training_data through the feature_func parameter.

feature_func: A function for generating feature layers that is applied to the data within the bounds of the input geometry. The feature_func must accept a dc_query dictionary, and return a single xarray.Dataset or xarray.DataArray containing 2D coordinates (i.e x, y - no time dimension). e.g.

    def feature_function(query):
        dc = datacube.Datacube(app='feature_layers')
        ds = dc.load(**query)
        ds = ds.mean('time')
        return ds

In [8]:
def feature_layers(dc, query):
    #load s2 annual geomedian
    ds = dc.load(product='gm_s2_annual',
                 **query)
    #calculate some band indices
    ds = calculate_indices(ds,
                           index=['NDVI', 'NDCI', 'IRECI', 'MTCI', 'OTCI', 'MCARI'
                                       , 'CI_RedEdge', 'CI_GreenEdge', 'TCARI', 'OSAVI', 'TCARI_OSAVI'],
                           drop=True,
                           satellite_mission='s2')
    
    return ds

Run the collect_training_data function

In [9]:
column_names, model_input = collect_training_data(
                                    gdf=input_data,
                                    dc=dc,
                                    dc_query=query,
                                    field=field,
                                    ncpus = ncpus,
                                    zonal_stats=zonal_stats,
                                    feature_func=feature_layers
                                    )

Taking zonal statistic: mean
Removed 0 rows wth NaNs &/or Infs
Output shape:  (402, 12)


In [10]:
print(column_names)
print('')
print(np.array_str(model_input, precision=2, suppress_small=True))

['class', 'NDVI', 'NDCI', 'IRECI', 'MTCI', 'OTCI', 'MCARI', 'CI_RedEdge', 'CI_GreenEdge', 'TCARI', 'OSAVI', 'TCARI_OSAVI']

[[4.   0.68 0.3  ... 0.18 0.58 0.31]
 [3.   0.51 0.19 ... 0.15 0.44 0.34]
 [3.   0.51 0.18 ... 0.12 0.44 0.28]
 ...
 [2.   0.63 0.27 ... 0.13 0.5  0.26]
 [2.   0.69 0.31 ... 0.16 0.57 0.28]
 [2.   0.74 0.34 ... 0.16 0.61 0.26]]


## Create traning datasets

In [11]:
#set the name and location of the output files
output_file = "results/training_data.csv"
output_file_txt = "results/training_data.txt"

In [12]:
model_col_indices = [column_names.index(var_name) for var_name in column_names]
#Export files to disk
np.savetxt(output_file, model_input[:, model_col_indices], header=", ".join(column_names),delimiter=',', fmt="%4f")

In [13]:
#grab all columns
model_col_indices = [column_names.index(var_name) for var_name in column_names]
#Export files to disk
np.savetxt(output_file_txt, model_input[:, model_col_indices], header=" ".join(column_names), fmt="%4f")